**Installing Transformers And Importing Dependencies**

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

**Load Summarization Pipeline**

In [ ]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access pu

config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

**Get Blog Post from Medium**

In [ ]:
URL = "https://hackernoon.com/will-the-game-stop-with-gamestop-or-is-this-just-the-beginning-2j1x32aa"

In [ ]:
r = requests.get(URL)

In [ ]:
r.text

'<!DOCTYPE html><html lang="en"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width"/><title>Will The Game Stop with Gamestop Or Is This Just The Beginning? | HackerNoon</title><meta name="description" content="Does the Game Stop with Gamestop? We discuss the broken price mechanism in a dangerous financial system."/><meta property="og:title" content="Will The Game Stop with Gamestop Or Is This Just The Beginning? | HackerNoon"/><meta property="og:description" content="Does the Game Stop with Gamestop? We discuss the broken price mechanism in a dangerous financial system."/><meta name="image" property="og:image" content="https://hackernoon.imgix.net/images/9ZkJ5JzVTQh7IupoflIpECnRHzB3-yg3s33cz.jpeg"/><meta property="twitter:title" content="Will The Game Stop with Gamestop Or Is This Just The Beginning? | HackerNoon"/><meta property="twitter:description" content="Does the Game Stop with Gamestop? We discuss the broken price mechanism in a dangerous financial sy

In [ ]:
soup = BeautifulSoup(r.text,"html.parser")
results = soup.find_all(["h1","p"])

In [ ]:
text = [result.text for result in results]
Article = " ".join(text)

In [ ]:
Article

'Will The Game Stop with Gamestop Or Is This Just The Beginning? The GameStop squeeze on short-sellers is an extraordinary event in markets, where at face value, retail traders and investors have worked together in an attempt to put some of the largest wall street institutions out of business. The events can be interpreted with many viable lenses and there are ironies baked in that are pure serendipity. There has been a centrally controlled game in the global financial system in which insiders benefited while outsiders got hurt that comes to a head with a company called GameStop. The broking firm of most of the retail side of this warfare ‘RobinHood’ is literally stealing from its poor, retail investors to give to its rich, capital backers. One of the historical realities of this game has been that macro-investing – the sages of not only portfolio management, but often also sophisticated social and cultural figures – have had a hard time making money in markets now for decades. With go

**Chunk Text**

In [ ]:
Article = Article.replace(".",".<eos>")
Article = Article.replace("!","!<eos>")
Article = Article.replace("?","?<eos>")
sentences = Article.split("<eos>")

In [ ]:
sentences[0]

'Will The Game Stop with Gamestop Or Is This Just The Beginning?'

In [ ]:
max_chunk = 500
current_chunk = 0
chunks = []
for sentence in sentences:
  if len(chunks) == current_chunk+1:
    if len(chunks[current_chunk]) + len(sentence.split(" ")) <= max_chunk:
      chunks[current_chunk].extend(sentence.split(" "))
    else:
      current_chunk += 1
      chunks.append(sentence.split(" "))
  else:
    print(current_chunk)
    chunks.append(sentence.split(" "))


0


In [ ]:
chunks[0]

['Will',
 'The',
 'Game',
 'Stop',
 'with',
 'Gamestop',
 'Or',
 'Is',
 'This',
 'Just',
 'The',
 'Beginning?',
 '',
 'The',
 'GameStop',
 'squeeze',
 'on',
 'short-sellers',
 'is',
 'an',
 'extraordinary',
 'event',
 'in',
 'markets,',
 'where',
 'at',
 'face',
 'value,',
 'retail',
 'traders',
 'and',
 'investors',
 'have',
 'worked',
 'together',
 'in',
 'an',
 'attempt',
 'to',
 'put',
 'some',
 'of',
 'the',
 'largest',
 'wall',
 'street',
 'institutions',
 'out',
 'of',
 'business.',
 '',
 'The',
 'events',
 'can',
 'be',
 'interpreted',
 'with',
 'many',
 'viable',
 'lenses',
 'and',
 'there',
 'are',
 'ironies',
 'baked',
 'in',
 'that',
 'are',
 'pure',
 'serendipity.',
 '',
 'There',
 'has',
 'been',
 'a',
 'centrally',
 'controlled',
 'game',
 'in',
 'the',
 'global',
 'financial',
 'system',
 'in',
 'which',
 'insiders',
 'benefited',
 'while',
 'outsiders',
 'got',
 'hurt',
 'that',
 'comes',
 'to',
 'a',
 'head',
 'with',
 'a',
 'company',
 'called',
 'GameStop.',
 '',
 '

In [ ]:
for chunk_id in range(len(chunks)):
  chunks[chunk_id] = " ".join(chunks[chunk_id])

In [ ]:
len(chunks[1].split(" "))

485

**Summarize Text**

In [ ]:
res = summarizer(chunks,max_length=120,min_length=30,do_sample=False)

Your max_length is set to 120, but your input_length is only 104. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=52)


In [ ]:
" ".join([summ["summary_text"] for summ in res])

' The GameStop squeeze on short-sellers is an extraordinary event in markets, where at face value, retail traders and investors have worked together in an attempt to put some of the largest wall street institutions out of business . The broking firm of most of the retail side of this warfare ‘RobinHood’ is stealing from its poor, retail investors to give to its rich, capital backers .  Victor Niederhoffer blew his hedge fund up in 1997 in a highly statistically improbable event, in which he sold puts that were targeted by market mechanics, rather than ‘truth’. The market here is more leveraged, more volatile, more aggressive, better for types of trading and worse for investing. The fundamentals no longer matter; and this was demonstrated only 5 years later with what could be the largest bubble of viable assets in history .  Greenspans moral hazard of late 1994/1995 fundamentally changed the market in a new way . Extraordinary people who were never wrong in risk-adjusted terms and who u

**Output In Text File**

In [ ]:
text = " ".join([summ["summary_text"] for summ in res])

In [ ]:
with open("blogsummary.tx","w") as f:
  f.write(text)